In [ ]:
# This is the generic code for health impact assessment (HIA) and burden of disease (BoD) calculations
# Forked originally from http://en.opasnet.org/w/HIA
# Original R code described in more detail at https://github.com/jtuomist/ghg-notebooks/wiki/Health-impact-assessment

import pandas as pd
import numpy as np
import json
import urllib.request
import math

#  sumExposcen subtracts the PAF of BAU scenario from the scenario of interest,
# thus giving the impact the exposure. The actual subtracting is done after the ovariable
# is evaluated by using CollapseMarginal (a standard function to manipulate ovariables in OpasnetUtils).

def sum_exposcen(out):
    if (pd.Series(['Exposcen']).isin(out.index.names).any()):
        out = out * pd.DataFrame({'Exposcen':['BAU','No exposure'], 'result':[1, -1]}).set_index('Exposcen')
        tmp = list(set(out.index.names) - {'Exposcen'})
        #ou = set(['Exposcen'])
        out = out.groupby(tmp).sum()
        
    return out

# fillna fills the NaN values of a column with all values available.
# object is pandas dataframe
# cols is a list of column names to fill

def fillna(object, cols):
    ind = object.index.names
    out = object.reset_index()
    if cols != [None]:
        for i in list(cols):
            a1 = out[out[i].notna()]
            a2 = out[out[i].isna()].drop(i, axis=1).assign(tmp=1)
            addition = pd.DataFrame({i:pd.unique(a1[i]), 'tmp':1})
            a2 = a2.merge(addition).drop('tmp',axis=1)
            out = a1.append(a2)
    if ind != [None]:
        out = out.set_index(ind)
    return out


gas = ['CO2','CH4']

In [ ]:
# Consumption is the amount of medium (food, water, air) consumed per time unit.

def consumption():
    df = pd.DataFrame({
        'matrix':['Food'],
        'result':[2]
    })
    return df.set_index('matrix')

# Concentration is the concentration of the exposure agent in the medium.

def concentration():
    df = pd.DataFrame({
        'matrix':['Food']*8,
        'exposure_agent':['PM2.5','Omega3','Fluoride','Aflatoxin','MeHg','campylobacter','norovirus','giardia'],
        'result':[1,2,3,4,5,6,7,8] # mg/kg, ppm, or similar
    })
    return df.set_index(['exposure_agent','matrix'])

# Exposure is the intensity of contact with the environment by the target population.

def exposure(
    p1 = concentration(),
    p2 = consumption()):
    
    out = p1 * p2
    #colnames(out@output)[colnames(out@output) == "Pathogen"] <- "Exposure_agent"

    return out

# bw is the body weight

def bw():
    df = pd.DataFrame({
        'sex':['male','female'],
        'result':[75,65]
    })
    return df.set_index('sex')

# Dose is the exposure scaled by logarithmic function or body weight

def dose(
    p1 = exposure(),
    p2 = bw()):
    
    outlog = np.log10(p1).assign(scaling='Log10').reset_index()

    outbw = p1.assign(tmp=1).set_index('tmp', append=True)
    outbw = outbw * p2.assign(tmp=1).set_index('tmp', append=True)
    outbw = outbw.assign(scaling='BW').reset_index().drop('tmp', axis=1)

    out = p1.reset_index().assign(scaling='None').append(outlog).append(outbw)
    out = out.set_index(['scaling','exposure_agent','sex','matrix']).sort_index()
    
    return fillna(out, ['sex'])


# Exposure-response function (ERF) is a variable that typically comes from data.
# Data comes from Opasnet [[ERFs of environmental pollutants]]
# http://en.opasnet.org/w/Special:Opasnet_Base?id=op_en5827
# The ovariables are converted to exposure_response_functions.csv by using code
# https://github.com/jtuomist/watch_network/create_erf_csv.R

def erf():
    df = pd.read_csv('exposure_response_functions.csv').drop(['source'], axis=1)
    
    tmp = ['er_function','observation','scaling','exposure_agent','response']
    tmp = tmp + list(set(df.columns) - set(tmp) - {'result'})
    
    df = df[tmp + ['result']].replace({
        'ERS':'UR', # Just one er_function name per equation
        'CSF':'UR',
        'OR':'RR', # treat odds ratio as risk ratio although it is close only at small risk levels
        'TWI':'Step',
        'TDI':'Step',
        'ADI':'Step',
        'RDI':'Step',
        'NOAEL':'Step'
    })
    df = df.sort_values(tmp).set_index(tmp)
    
    return df


# Frexposed is the fraction of exposed individuals within the target population. Defaults to 1
# but may be indexed by population subgroups.

def frexposed():
    df = pd.DataFrame({
        'sex':['male','female'],
        'result':[1,1]
    })
    return df.set_index('sex')

# P_illness is the probability of illness. Relevant for microbial infection endpoints.
# Typically a microbe-specific constant.

def p_illness():
    df = pd.DataFrame({
        'response':['campylobacter infection','giardia infection','norovirus infection'],
        'result':[1,1,1]
    })
    return df.set_index('response')

# Relative risk (RR) is the risk of an exposed individual compared with a counterfactual
# unexposed individual using the modelled exposures. 

p1 = pd.DataFrame({
    'Exposcen':['BAU','BAU','No exposure','No exposure'],
    'sex':['male','female','male','female'],
    'result':[4, 4, 2, 3]}).set_index(['Exposcen','sex'])

## Relative risk (RR) is the risk of exposed divided by risk of unexposed

def rr(
    p1 = erf(),
    p2 = dose()):

    RR = p1.loc[('RR','ERF')] 
    threshold = p1.loc[('RR','Threshold')]
    dose2 = (p2 - threshold).dropna()
    dose2['result'] = np.clip(dose2['result'], 0, None) # Smallest allowed value is 0

    t1 = np.exp(np.log(RR) * dose2).dropna()
    
    Imax = p1.loc[('Relative Hill','ERF')]
    ed50 = p1.loc[('Relative Hill','Threshold')]
    
    t2 = (1 + (p2 * Imax) / (p2 + ed50)).dropna()
    
    out = t1.reset_index().append(t2.reset_index())
    
    keep = list(out.columns[out.assign(scaling=None).notna().any()])
    out = out[keep].set_index(list(set(keep) - {'result'}))

    return out

def incidence():
    df = pd.DataFrame({
        'response':['Liver cancer','Fluorosis','MeHg TWI','campylobacter infection',
                    'norovirus infection','giardia infection'] + ["Loss in child's IQ points"]*3,
        'sex':[None]*7 + ['male','female'],
        'result':[1,1,1,1,1,1,1,1,1]
    })
    df = df.set_index(['response','sex'])
    return fillna(df, ['sex'])


## Population attributable fraction PAF

def paf(
    p1 = erf(),
    p2 = dose(),
    p3 = frexposed(),
    p4 = incidence(),
    p5 = rr(),
    p6 = p_illness()):
    
    k = p1.loc[('UR','ERF')]
    threshold = p1.loc[('UR','Threshold')]
    dose2 = (p2 - threshold).dropna()
    dose2['result'] = np.clip(dose2['result'], 0, None) # Smallest allowed value is 0
    out1 = (k * dose2 * p3 / p4).dropna()
    
    upper = p1.loc[('Step','ERF')]
    lower = p1.loc[('Step','Threshold')]
    out2 = 1 - (((p2 - lower).dropna() >= 0) & ((p2 - upper).dropna() <= 0)).astype(int)
    out2 = (out2 * p3 / p4).dropna()
    
    out3 = p3.assign(tmp=1).set_index('tmp', append=True) * (p5.assign(tmp=1).set_index('tmp', append=True) - 1)
    r = out3['result']
    out3['result'] = (r > 0).astype(int) * (r/(r + 1)) + (r <= 0).astype(int) * r
    out3 = out3.reset_index('tmp').drop('tmp',axis=1)
    
    param1 = p1.loc[('beta poisson approximation','ERF')]
    param2 = p1.loc[('beta poisson approximation','Threshold')]
    out4 = ((1 - (1 + p2/param2)**(-1 * param1)) * p3).dropna()
    out4 = (out4 / p4 * p6).dropna() # dropna is needed before an index with NaN is used for merging

    param1 = p1.loc[('exact beta poisson','ERF')]
    param2 = p1.loc[('exact beta poisson','Threshold')]
    out5 = ((1 - np.exp(-1 * param1/(param1 + param2) * p2)) * p3).dropna()
    out5 = (out5 / p4 * p6).dropna()
    
    k = p1.loc[('exponential','ERF')]
    out6 = ((1 - np.exp(-1 * k * p2)) * p3).dropna()
    out6 = (out6 / p4 * p6).dropna()

    out = out1.reset_index().append(out2.reset_index()).append(out3.reset_index())
    out = out.append(out4.reset_index()).append(out5.reset_index()).append(out6.reset_index())
    out = out.drop(['scaling','matrix','exposure','exposure_unit'],axis=1)

    keep = set(out.columns[out.notna().any()]) # remove indices that are empty
    fill = set(out.columns[out.isna().any()]) # fill indices that have some empty locations
    out = fillna(out, list(fill.intersection(keep) - {'result'}))
    out = out[list(keep)].set_index(list(keep - {'result'}))
    
    return out

# Population is typically indexed by subgroups.

def population():
    df = pd.DataFrame({
        'age':['<14','<14','>=14','>=14'],
        'sex':['male','female','male','female'],
        'result':[100000]*4
    })
    return df.set_index(['age','sex'])


# Case_burden is the disease burden of a single case of disease. This may be indexed by population subgroup e.g. age.

def case_burden():
    df = pd.DataFrame({
        'response':['Fluorosis', 'Liver cancer', "Loss in child's IQ points",
       'MeHg TWI', 'Breast cancer', 'CHD2 mortality', 'CHD3 mortality',
       'Coronary heart disease mortality', 'Stroke mortality',
       'Cardiopulmonary mortality', 'Lung cancer mortality',
       'Total mortality', 'CHD arrythmia mortality',
       'campylobacter infection', 'norovirus infection',
       'giardia infection'],
        'result':[3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
    })
    return df.set_index(['response'])


# BoD is the current (observed) burden of disease (measured in disability-adjusted life years or DALYs).

def bod(
    p1 = incidence(),
    p2 = population(),
    p3 = case_burden()):

    out = p1 * p2 * p3

    return out


# bod_attr is the burden of disease that can be attributed to the exposure of interest.

def bod_attr(
    p1 = bod(),
    p2 = paf()):

    out = p1 * p2
    
    return out.dropna()

display(bod_attr())

In [ ]:
# mc2d function is not needed until we start using Monte Carlo and Iter in multi-index

"""
    mc2d is a function that samples the ovariable that describes individuals and then aggregates (typically averages over) the samples to reflect the situation of a defined population or population subgroups. This is done because in disease burden assessments, we are typically interested in population-level uncertainties rather than individual uncertainties.

    mc2d
    function (ova, mc2dpar = NULL)
    {
    if (is.null(mc2dpar))
    if (exists("mc2dparam"))
    mc2dpar <- mc2dparam
    else stop("Parameter list mc2dparam missing!\n")
    if (mc2dpar$run2d) {
    ova <- ova * mc2dpar$info
    require(reshape2)
    marg <- setdiff(c(colnames(ova@output)[ova@marginal],
    mc2dpar$newmarginals), "Iter")
    out <- aggregate(result(ova), by = ova@output[colnames(ova@output) %in%
    marg], FUN = function(x) {
    strength <- if (is.null(mc2dpar$strength))
    length(x)
    else mc2dpar$strength
    apply(array(as.numeric(sample(as.character(x), strength *
    mc2dpar$N2, replace = TRUE)), dim = c(strength,
    mc2dpar$N)), MARGIN = 2, FUN = mc2dpar$fun)
    })
    temp <- melt(out[[length(out)]])
    out[[length(out)]] <- 1:nrow(out)
    colnames(temp) <- c("Nrow", "Iter", "Result")
    out <- merge(out, temp, by.x = "x", by.y = "Nrow")
    out$x <- NULL
    out <- Ovariable(output = out, marginal = colnames(out) %in%
    c(marg, "Iter"))
    }
    else {
    out <- ova
    }
    return(out)
    }

"""
